In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
torch.manual_seed(1234)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-14B-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-14B-Chat", device_map="cuda", trust_remote_code=True).eval()


In [ ]:
def describe_image(image_path, seed=4):
    try:
        with torch.no_grad():
            torch.manual_seed(seed)
            query = tokenizer.from_list_format([
                {'image': image_path},
                {'text': "这是什么？请示出 - 产品品类： ； 品牌： ；副品牌： ；产品名称： ； 净含量： ； 颜色： ；"},
                {'text': 'Describe this image with a rich description. If possible, identify the product category, brand, subbrand, container type and SKU name. Give your answer in this format: Description: ; Category: ; Brand: ; Sub-brand: ; SKU Name: '},
            ])
            response, history = model.chat(tokenizer, query=query, history=None)
            return response
    except:
        print(image_path)
        return image_path

In [ ]:
import os
folder_path = "all" #path to folder with images e.g. all/train/product_folder/image.jpg
filepath_list = []
for folder in os.listdir(folder_path):
    for product_folder in os.listdir(folder_path+"/"+folder):
        filepath_list.extend([folder_path+"/"+folder+"/"+ product_folder + "/" + x for x in os.listdir(folder_path+"/"+folder+"/"+product_folder)])
df = pd.DataFrame(filepath_list)
df.columns = ['filepath']
df.to_csv("filepaths.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("filepaths.csv")
products = df.filepath.apply(lambda x: x.split("/")[2]).values
df['product'] = products
df = df[df['filepath'].apply(lambda x: "others" in x)]
df.to_excel("output.xlsx", index=False)

In [ ]:
df = pd.read_excel("output.xlsx")
df['qwen_description'] = df['filepath'].progress_apply(describe_image)
df.to_excel("output_qwen.xlsx", index=False)